IMPORT LIBRARIES AND DATA PREPROCESSING

In [39]:
import pandas as pd
import zipfile
import regex as re
from os import listdir

In [31]:
!curl -O https://md-datasets-cache-zipfiles-prod.s3.eu-west-1.amazonaws.com/7zbk8zsd8y-1.zip
with zipfile.ZipFile("7zbk8zsd8y-1.zip") as zip_ref:
    zip_ref.extractall("dataset/")
!rm 7zbk8zsd8y-1.zip -r

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 65220  100 65220    0     0  24940      0  0:00:02  0:00:02 --:--:-- 24931
rm: -r: No such file or directory


In [69]:
allData = []
#Listing all the datasets
for paths in listdir('dataset/'):
    with open('dataset/' + paths, 'rb') as readFile:
        data = readFile.read()
        allData.append(data)


In [71]:
for i in range(len(allData)):
  allData[i] = allData[i].decode('latin-1')

In [77]:
#Combining all the files
txt = "\n".join(allData)
data = txt.splitlines()
len(data)

1742

DATA PROCESSING

In [78]:
#Storing into dataset.txt by joining all the textfields.
with open("dataset.txt", "w") as file:
    file.write("\n".join(data))
     

In [143]:
data = ""

#Read from dataset.txt
with open("dataset.txt", "r") as file:
    txt = file.read()
    data = txt.splitlines()
    print(len(data))

data[:5]

1742


['As a repository manager, I want to know all the collections and objects in the DAMS for which I have custodial responsibility. ',
 'As a repository manager, I want to be able to update repository information in one place and be assured updates will be propagated to all relevant places. ',
 'As a repository manager, I want to constrain curatorial access to my units assets to staff members of my unit and to DAMS managers having universal access. ',
 'As a user, I want to list all the collections and objects in the DAMS that belong to a library unit. ',
 'As a user, I want to be able to contact repositories. ']

In [144]:
#Storage - Dataframe - Defining the structure
df = pd.DataFrame(columns = ["Stakeholders", "Stories"])

In [145]:
#Removing as a from the stakeholders
shList = []
storyList = []
for i in data:
  i = str(i)
  if i!= None: 
    if re.search(r'As a(?:n)? (.*?)\,', i):
      roles = re.search(r'As a(?:n)? (.*?)\,', i).group(1)
      if re.search(r'\, (.*?)\.', i):
        story = re.search(r'\, (.*?)\.', i).group(1)
      else:
        story = ""
    shList.append(roles.title())
    storyList.append(story)

In [146]:
#Storing into Dataframe
df["Stakeholders"] = shList
df["Stories"] = storyList
df.head()

,Stakeholders,Stories
0,Repository Manager,I want to know all the collections and objects...
1,Repository Manager,I want to be able to update repository informa...
2,Repository Manager,I want to constrain curatorial access to my un...
3,User,I want to list all the collections and objects...
4,User,I want to be able to contact repositories


In [147]:
#Convert to strings to avoid errors
df['Stakeholders'] = df['Stakeholders'].astype(str)
df['Stories'] = df['Stories'].astype(str)

In [152]:
#Aggregating the stakeholders to make stories for each
df2 = df.groupby('Stakeholders').agg(". ".join)
df2 = df2.reset_index()

In [154]:
#Displaying the result of First Problem
df2.head()

,Stakeholders,Stories
0,Academicpublisher,I want to make persistent web links between my...
1,Academicresearcher,I want to find the works of a particular autho...
2,Admin,"I want to see, validate and edit segmentation ..."
3,Administrator,"I want to create user groups, so that I can co..."
4,Advertiser,I want to be emailed a reminder seven day befo...
